In [51]:
import logging
import elasticsearch
from nltk.tokenize import TreebankWordTokenizer as twt

def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'ef43c3710dcb', 'port': 9200}])
    if _es.ping():
        print('Connected to Elastic')
    else:
        print('Not connected')
    return _es

es = connect_elasticsearch()

Connected to Elastic


In [100]:
import pickle

crf = pickle.load(open( "crf_model-with-reg.pkl", "rb" ) )

In [58]:
import re

def matchesRegEx(regex, word):
    res = re.match(regex, word)
    if res:
        return True
    return False

def gazeteerMoney(word):
    moneyDictionary = ['лв', 'евро', 'долара', 'лева', '$', 'щ. д.', 'щ.д.']
    for w in moneyDictionary:
        res = re.match(w, word)
        if res:
            return True
    return False

def gazeteerDate(word):
    moneyDictionary = ['месец', 'януари', 'февруари', 'март', 'април', 'май', 'юни', 'юли', 'август', 'септември', 'октомври', 'ноември', 'декември', 'година', 'год.', 'г.']
    for w in moneyDictionary:
        res = re.match(w, word)
        if res:
            return True
    return False

def gazeteerCourt(word):
    dictionary = ["PC", "ОС", "СГС", "АДМС", "АС", "ВКС", "ВАС", 'СЪД', 'ЕСПЧ']
    for w in dictionary:
        res = re.match(w, word.upper())
        if res:
            return True
    return False
    
def gazeteerJudge(word):
    moneyDictionary = ['съд.', 'съдия']
    for w in moneyDictionary:
        res = re.match(w, word)
        if res:
            return True
    return False

def gazeteerInstitution(word):
    moneyDictionary = ['БНТ', 'съдия', 'ЕС', 'КЗК', 'МВР', 'НАП', ]
    for w in moneyDictionary:
        res = re.match(w, word)
        if res:
            return True
    return False

def gazeteerDocRef(word):
    moneyDictionary = ['бел', 'бр', 'ДВ', "к.д.", ]
    for w in moneyDictionary:
        res = re.match(w, word)
        if res:
            return True
    return False

def gazeteerJudge(word):
    moneyDictionary = ['съд.', 'съдия']
    for w in moneyDictionary:
        res = re.match(w, word)
        if res:
            return True
    return False

def gazeteerCountry(word):
    moneyDictionary = ['РБ', 'ГДР', 'КРБ', 'България', 'Сърбия', 'Румъния', 'САЩ']
    for w in moneyDictionary:
        res = re.match(w, word)
        if res:
            return True
    return False

def gazeteerLaw(word):
    moneyDictionary = ['чл', 'ал', 'алинея', 'т', "НК", "НПК", "ГПК", "АПК", "СК", "ЗАНН", "ЗГС", "ЗЗД", "ЗЗК", "ЗЗП", "ЗИДАПК", "ЗИД", "ЗНА", "ЗРТ", "ЗХр", "ЗСВ"]
    for w in moneyDictionary:
        res = re.match(w, word)
        if res:
            return True
    return False

def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[:-2]': word[:-2],
        'word[:-1]': word[:-1],
        'word[-1:]': word[-1:],
        'word[-2:]': word[-2:],
        'ispunctuation': matchesRegEx("[,\.;:\?!-+\"]", word),
        'containsDash': matchesRegEx(".*-.*", word),
        'containsDot': matchesRegEx(".*\..*", word),
        'containsDigit': matchesRegEx(".*[0-9].*", word),
        'lonleyInitial': matchesRegEx("[А-Я]\.", word),
        'singleChar': matchesRegEx("[А-Яа-я]", word),
        'singleDigit': matchesRegEx("[0-9]", word),
        'hasQuote': matchesRegEx("[\"']", word),
        'endsWithDot': matchesRegEx(".*\.", word),
        'fourDigitsYear': matchesRegEx("\d{4}", word),
        'gazeteerMoney': gazeteerMoney(word),
        'gazeteerJudge': gazeteerJudge(word),
        'gazeteerCountry': gazeteerCountry(word),
        'gazeteerInstitution': gazeteerInstitution(word),
        'gazeteerDocRef': gazeteerDocRef(word),
        'gazeteerDate': gazeteerDate(word),
        'gazeteerCourt': gazeteerCourt(word),
        'gazeteerLaw': gazeteerLaw(word),
        'word.isupper()': word.isupper(),
        'word.islower()': word.islower(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:ispunctuation': matchesRegEx("[,\.;:\?!-+\"]", word1),
            '-1:containsDash': matchesRegEx(".*-.*", word1),
            '-1:containsDot': matchesRegEx(".*\..*", word1),
            '-1:containsDigit': matchesRegEx(".*[0-9].*", word1),
            '-1:singleDigit': matchesRegEx("[0-9]", word1),
            '-1:endsWithDot': matchesRegEx(".*\.", word1),
            '-1:gazeteerMoney': gazeteerMoney(word),
            '-1:gazeteerJudge': gazeteerJudge(word),
            '-1:gazeteerCountry': gazeteerCountry(word),
            '-1:gazeteerInstitution': gazeteerInstitution(word),
            '-1:gazeteerDocRef': gazeteerDocRef(word),
            '-1:lonleyInitial': matchesRegEx("[А-Я]\.", word1),
            '-1:singleChar': matchesRegEx("[А-Яа-я]", word1),
            '-1:fourDigitsYear': matchesRegEx("\d{4}", word1),
            '-1:gazeteerMoney': gazeteerMoney(word1),
            '-1:gazeteerDate': gazeteerDate(word1),
            '-1:gazeteerCourt': gazeteerCourt(word1),
            '-1:gazeteerLaw': gazeteerLaw(word1),
            '-1:hasQuote': matchesRegEx("[\"']", word1),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:ispunctuation': matchesRegEx("[,\.;:\?!-+\"]", word1),
            '+1:containsDash': matchesRegEx(".*-.*", word1),
            '+1:containsDot': matchesRegEx(".*\..*", word1),
            '+1:containsDigit': matchesRegEx(".*[0-9].*", word1),
            '+1:singleDigit': matchesRegEx("[0-9]", word1),
            '+1:endsWithDot': matchesRegEx(".*\.", word1),
            '+1:lonleyInitial': matchesRegEx("[А-Я]\.", word1),
            '+1:singleChar': matchesRegEx("[А-Яа-я]", word1),
            '+1:fourDigitsYear': matchesRegEx("\d{4}", word1),
            '+1:gazeteerMoney': gazeteerMoney(word),
            '+1:gazeteerJudge': gazeteerJudge(word),
            '+1:gazeteerCountry': gazeteerCountry(word),
            '+1:gazeteerInstitution': gazeteerInstitution(word),
            '+1:gazeteerDocRef': gazeteerDocRef(word),
            '+1:gazeteerMoney': gazeteerMoney(word1),
            '+1:gazeteerDate': gazeteerDate(word1),
            '+1:gazeteerCourt': gazeteerCourt(word1),
            '+1:gazeteerLaw': gazeteerLaw(word1),
            '+1:hasQuote': matchesRegEx("[\"']", word1)
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [76]:
test = """ОПРЕДЕЛЕНИЕ

№ 11137
София, 26.10.2015

Върховният административен съд на Република България - Второ отделение, в закрито заседание в състав:

ПРЕДСЕДАТЕЛ:
СВЕТЛАНА ЙОНКОВА
ЧЛЕНОВЕ:
НАДЕЖДА ДЖЕЛЕПОВА
ТАНЯ РАДКОВА
при секретар		
и с участието
на прокурора		
изслуша докладваното
от съдията	ТАНЯ РАДКОВА	
по адм. дело № 9978/2015. Document Link Icon


Производството е по чл. 229 и следващите АПК.
Образувано е по частна жалба на И. К. М., Р. Н. Щ.-Пещинска и К. Р. П. срещу определение, постановено в открито съдебно заседание на 30.07.2015 г. по адм. д. № 163/2015 г. на Административен съд Смолян. Считат обжалваното определение за неправилно и претендират отмяната му.
В срока по чл. 232 АПК другата страна не взема становище по частната жалба.
Частната жалба е подадена в срок и е процесуално допустима.
Разгледана по същество, е основателна при следните съображения:
Частните жалбоподатели са подали заявление до административния орган за издаване на скица-проект на недвижим имот, необходим за снабдяване с констативен нотариален акт. Към заявлението е приложен проект за изменение на кадастралната карта и регистър съобразно чл. 52 ЗКИР. С писмо № 24-8320/02.06.2015 г. на началника на службата по геодезия, картография и кадастър С. им се съобщава, че претендираната от тях за издаване скица- проект ще бъде издадена след провеждане на административна процедура по изменение на кадастралната карта и регистъра по чл. 54 ЗКИР. Това писмо е предмет на обжалване пред първоинстанционния съд. С обжалваното определение първоинстанционният съд е приел, че в случая не се касае до отказ за издаване на скица или скица-проект, а до писмо с уведомителен характер. Поради това е оставил жалбата без разглеждане и е прекратил производството по делото.
Определението е незаконосъобразно:
В случая писмото е индивидуален административен акт в хипотезата на отказ. За характера на акта следва да се съди по съдържащото в него волеизявление. С конкретното в настоящия казус по същество се отказва издаването на скица-проект по подаденото заявление, като без да има претенция от заявителите се инициира от административния орган процедура по изменение на кадастралната карта и регистър поради грешка или непълнота. Такава претенция не е заявявана от жалбоподателите и не удовлетворява тяхното заявление. Поради това обжалваното писмо е с характер на отказ и представлява подлежащ на обжалване индивидуален административен акт. Това налага обжалваното определение да бъде отменено, а делото да се върне на същия съд за продължаване на съдопроизводствените действия.
Воден от горното, Върховният административен съд, второ отделение,

ОПРЕДЕЛИ:

ОТМЕНЯ определение, постановено в открито съдебно заседание на30.07.2015 г. по адм. д. № 163/2015 г. на Административен съд Смолян, с което е прекратено производството по делото.
ВРЪЩА делото на същия съд за продължаване на съдопроизводствените действия съобразно указанията в настоящето определение.
Определението не подлежи на обжалване.


Особено мнение:"""

In [77]:
test_text_features = sent2features([(t, None) for t in twt().tokenize(test)])
res = crf.predict([test_text_features])
list(zip(twt().tokenize(test), res[0]))

[('ОПРЕДЕЛЕНИЕ', 'O'),
 ('№', 'O'),
 ('11137', 'O'),
 ('София', 'O'),
 (',', 'O'),
 ('26.10.2015', 'O'),
 ('Върховният', 'O'),
 ('административен', 'O'),
 ('съд', 'O'),
 ('на', 'O'),
 ('Република', 'LOC-CTRY'),
 ('България', 'LOC-CTRY'),
 ('-', 'O'),
 ('Второ', 'O'),
 ('отделение', 'O'),
 (',', 'O'),
 ('в', 'O'),
 ('закрито', 'O'),
 ('заседание', 'O'),
 ('в', 'O'),
 ('състав', 'O'),
 (':', 'O'),
 ('ПРЕДСЕДАТЕЛ', 'O'),
 (':', 'O'),
 ('СВЕТЛАНА', 'O'),
 ('ЙОНКОВА', 'O'),
 ('ЧЛЕНОВЕ', 'O'),
 (':', 'O'),
 ('НАДЕЖДА', 'O'),
 ('ДЖЕЛЕПОВА', 'O'),
 ('ТАНЯ', 'O'),
 ('РАДКОВА', 'O'),
 ('при', 'O'),
 ('секретар', 'O'),
 ('и', 'O'),
 ('с', 'O'),
 ('участието', 'O'),
 ('на', 'O'),
 ('прокурора', 'O'),
 ('изслуша', 'O'),
 ('докладваното', 'O'),
 ('от', 'O'),
 ('съдията', 'O'),
 ('ТАНЯ', 'O'),
 ('РАДКОВА', 'O'),
 ('по', 'O'),
 ('адм.', 'O'),
 ('дело', 'O'),
 ('№', 'O'),
 ('9978/2015.', 'LOC-OTH'),
 ('Document', 'LOC-OTH'),
 ('Link', 'LOC-OTH'),
 ('Icon', 'LOC-OTH'),
 ('Производството', 'LOC-OTH'),
 (

In [120]:
## ToDO: use utils
def get_spans(token_list):
    current_offset = 0
    spans = []
    for token in token_list:
        if token[1] != 'O':
            spans.append([current_offset, current_offset+len(token[0])-1, token[1], token[0]])
        current_offset += len(token[0]) + 1 # plus delimiter
    return spans

## TODO: use utils
def merge_adjacent_labels(labels):
    merged_labels = []
    prev_label = None
    new_label = None
    
    for label in labels:
        if prev_label == None:
            new_label = [label[0], label[1], label[2], label[3]]
            prev_label = label
            continue
        
        if prev_label[1] + 2 == label[0] and prev_label[2] == label[2]:
            new_label[1] = label[1]
            new_label[3] += " " + label[3]
        else:
            merged_labels.append({
                'offset_start': new_label[0],
                'offset_end': new_label[1],
                'type': new_label[2],
                'text': new_label[3],
            })
            new_label = [label[0], label[1], label[2], label[3]]
        prev_label = label
    return merged_labels

def group_by_entity_type(entities_span_list):
    entities_dict = {}
    
    for ent in entities_span_list:
        if ent['type'] not in entities_dict:
            entities_dict[ent['type']] = []
            entities_dict[ent['type']].append(ent['text'])
        else:
            entities_dict[ent['type']].append(ent['text'])
            
    return entities_dict

class ESDoc:
    entities = {}
    entities_spans = []
    keywords = []
    doc = None
    
    def __init__(self, doc, keywords=[]):
        self.doc = doc
        self.keywords = keywords
        test_text_features = sent2features([(t, None) for t in twt().tokenize(doc)])
        res = crf.predict([test_text_features])
        entities = list(zip(twt().tokenize(doc), res[0]))
        self.entities_spans = get_spans(entities)
        self.entities_spans = merge_adjacent_labels(self.entities_spans)
        self.entities = group_by_entity_type(self.entities_spans)

In [121]:
t = ESDoc(test)
t.entities_spans

[{'offset_start': 73,
  'offset_end': 90,
  'type': 'LOC-CTRY',
  'text': 'Република България'},
 {'offset_start': 423,
  'offset_end': 430,
  'type': 'PER-OTH',
  'text': 'И. К. М.'},
 {'offset_start': 454,
  'offset_end': 461,
  'type': 'PER-OTH',
  'text': 'К. Р. П.'},
 {'offset_start': 526,
  'offset_end': 535,
  'type': 'DATE',
  'text': '30.07.2015'},
 {'offset_start': 568,
  'offset_end': 601,
  'type': 'ORG-COURT',
  'text': 'Административен съд Смолян. Считат'},
 {'offset_start': 679,
  'offset_end': 689,
  'type': 'REF-LAW',
  'text': 'чл. 232 АПК'},
 {'offset_start': 1154,
  'offset_end': 1171,
  'type': 'DATE',
  'text': '24-8320/02.06.2015'},
 {'offset_start': 2583,
  'offset_end': 2612,
  'type': 'ORG-COURT',
  'text': 'Върховният административен съд'}]

In [112]:
documents = []

In [113]:
from IPython.core.display import HTML
import json

CRAWLER_DATA_FILE = '../law_dataset_crawler/out.json'
with open(CRAWLER_DATA_FILE, 'r') as crawler_data_file:
    data = json.load(crawler_data_file)
    
import re

def clean_legal_text(text):
    text_copy = text
    
    try:
        num = re.search('(№)\s+(\d*)', text_copy)
        text_copy = re.sub('(№)\s+(\d*)', num.group(1)+num.group(2), text_copy)
    except:
        pass
    
    regex_rules = [
        (r'(?:(?!\n)\s){1,5}', ' '),
        (r'<!--[^>]*-->', ''),
        (r'(?:(?!\n)\s)(?:(?!\n)\s)+', '\n')
    ]
    
    for rule in regex_rules:
        text_copy = re.sub(rule[0], rule[1], text_copy)
    
    rules = [
        ('Р Е Ш Е Н И Е', 'РЕШЕНИЕ'),
        ('И М Е Т О', 'ИМЕТО'),
        ('Н А', 'НА'),
        ('Н А Р О Д А', 'НАРОДА'),
        ('В ИМЕТО НА НАРОДА', 'В ИМЕТО НА НАРОДА\n'),
        ('Р Е Ш И', 'РЕШИ'),
        ('О П Р Е Д Е Л И', 'ОПРЕДЕЛИ'),
        ('О П Р Е Д Е Л Е Н И Е', 'ОПРЕДЕЛЕНИЕ')
    ]
    
    for rule in rules:
        text_copy = text_copy.replace(rule[0], rule[1])
        
    return text_copy


In [129]:
def create_index(es_object, index_name='law_dataset'):
    created = False
    # index settings
    settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "court_decisions": {
                "dynamic": "strict",
                "properties": {
                    "entities": {
                        "type": "nested",
                        "properties": {
                            'offset_start': {'type': 'integer'},
                            'offset_end': {'type': 'integer'},
                            'type': {'type': 'text'},
                            'text': {'type': 'text'}
                        }
                    },
                    "entities_spans": {
                        "type": "nested",
                    },
                    "keywords": {
                        "type": "nested"
                    },
                    "doc": {
                        "type": "text"
                    },
                }
            }
        }
    }
    try:
        if not es_object.indices.exists(index_name):
            # Ignore 400 means to ignore "Index Already Exist" error.
            es_object.indices.create(index=index_name, ignore=400, body=settings)
            print('Created Index')
        created = True
    except Exception as ex:
        print(str(ex))
    finally:
        return created

In [128]:
create_index(es, 'law_dataset')

True

In [134]:
import jsonlines
from bs4 import BeautifulSoup

success = 0
fails = 0

MAX_ITERS = len(data)
for i in range(MAX_ITERS):
    soup = BeautifulSoup(data[i]['text'])
    keywords_text = soup.body.table.contents[1].getText()
    keywords_list = [keyword.strip() for keyword in keywords_text.split("*")]

    contents = soup.find_all("table")[1].getText()
    cleaned_legal_text = clean_legal_text(contents)
    
    doc = ESDoc(cleaned_legal_text, keywords_list)
    #  documents.append(ESDoc(cleaned_legal_text, keywords_list))
    
    ## Create ES index for that document
    res = es.index(index='law_dataset', doc_type='court_decisions', body=doc.__dict__)
    print(res)
    if res['result'] == 'created':
        success += 1
    else:
        fails += 1
        
    print("Success: " + str(success) + " Fails: " + str(fails) + "From: " + str(MAX_ITERS))

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'j9pSFXAB0PUA6disXzHp', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
Success: 1 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'kNpSFXAB0PUA6disZjHO', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
Success: 2 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'kdpSFXAB0PUA6disazFF', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
Success: 3 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'ktpSFXAB0PUA6discjF4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
Success: 4 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': '

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'sdpTFXAB0PUA6disBjH6', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 34, '_primary_term': 1}
Success: 35 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'stpTFXAB0PUA6disCDHe', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 35, '_primary_term': 1}
Success: 36 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 's9pTFXAB0PUA6disDzEM', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 36, '_primary_term': 1}
Success: 37 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'tNpTFXAB0PUA6disEjFw', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 37, '_primary_term': 1}
Success: 38 Fails: 0From: 15779
{'_index': 'law_dataset', '_

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': '09pTFXAB0PUA6disdjHe', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 68, '_primary_term': 1}
Success: 69 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': '1NpTFXAB0PUA6diseTFt', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 69, '_primary_term': 1}
Success: 70 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': '1dpTFXAB0PUA6disezEN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 70, '_primary_term': 1}
Success: 71 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': '1tpTFXAB0PUA6disfDGp', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 71, '_primary_term': 1}
Success: 72 Fails: 0From: 15779
{'_index': 'law_dataset', '_

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': '9dpTFXAB0PUA6dis7DFs', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 102, '_primary_term': 1}
Success: 103 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': '9tpTFXAB0PUA6dis9TG-', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 103, '_primary_term': 1}
Success: 104 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': '99pTFXAB0PUA6dis-jFJ', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 104, '_primary_term': 1}
Success: 105 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': '-NpTFXAB0PUA6dis-zHi', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 105, '_primary_term': 1}
Success: 106 Fails: 0From: 15779
{'_index': 'law_data

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'F9pUFXAB0PUA6diskDLu', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 136, '_primary_term': 1}
Success: 137 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'GNpUFXAB0PUA6diskzLx', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 137, '_primary_term': 1}
Success: 138 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'GdpUFXAB0PUA6dissTJX', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 138, '_primary_term': 1}
Success: 139 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'GtpUFXAB0PUA6disuTLC', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 139, '_primary_term': 1}
Success: 140 Fails: 0From: 15779
{'_index': 'law_data

...

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'H9raFXAB0PUA6disv070', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7312, '_primary_term': 1}
Success: 7313 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'INraFXAB0PUA6disx07Y', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7313, '_primary_term': 1}
Success: 7314 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'IdraFXAB0PUA6disyk7V', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7314, '_primary_term': 1}
Success: 7315 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'ItraFXAB0PUA6diszU5C', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7315, '_primary_term': 1}
Success: 7316 Fails: 0From: 15779
{'_index': '

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'QdrbFXAB0PUA6disMU4N', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7346, '_primary_term': 1}
Success: 7347 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'QtrbFXAB0PUA6disMk5l', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7347, '_primary_term': 1}
Success: 7348 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'Q9rbFXAB0PUA6disNE4-', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7348, '_primary_term': 1}
Success: 7349 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'RNrbFXAB0PUA6disNk5C', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7349, '_primary_term': 1}
Success: 7350 Fails: 0From: 15779
{'_index': '

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'Y9rbFXAB0PUA6dispU7B', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7380, '_primary_term': 1}
Success: 7381 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'ZNrbFXAB0PUA6disqE47', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7381, '_primary_term': 1}
Success: 7382 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'ZdrbFXAB0PUA6disq05C', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7382, '_primary_term': 1}
Success: 7383 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'ZtrbFXAB0PUA6disrk5A', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7383, '_primary_term': 1}
Success: 7384 Fails: 0From: 15779
{'_index': '

{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'hdrcFXAB0PUA6disTE5C', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7414, '_primary_term': 1}
Success: 7415 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'htrcFXAB0PUA6disUU7O', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7415, '_primary_term': 1}
Success: 7416 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'h9rcFXAB0PUA6disVk42', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7416, '_primary_term': 1}
Success: 7417 Fails: 0From: 15779
{'_index': 'law_dataset', '_type': 'court_decisions', '_id': 'iNrcFXAB0PUA6disYU5k', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7417, '_primary_term': 1}
Success: 7418 Fails: 0From: 15779
{'_index': '

KeyboardInterrupt: 